In [42]:
# Import dependencies

import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

# Python SQL toolkit and Object Relational Mapper
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import psycopg2
# Database credentials
import config


# Extract data from database

In [2]:
con_string=f'postgresql+psycopg2://{config.username}:{config.password}@{config.endpoint}:5432/superstoredatabase'

In [3]:
engine = create_engine(con_string)

In [4]:
session = Session(engine)

In [11]:
superstore_df = pd.read_sql("SELECT * FROM superstore_complete", engine)
superstore_df.head(10)


,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,...,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,person,return
0,32298,CA-2012-124891,31/07/2012,31/07/2012,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical,Kelly Williams,None
1,26341,IN-2013-77878,05/02/2013,07/02/2013,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical,Anthony Jacobs,Yes
2,25330,IN-2013-71249,17/10/2013,18/10/2013,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium,Anthony Jacobs,None
3,13524,ES-2013-1579342,28/01/2013,30/01/2013,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium,Anna Andreadi,None
4,47221,SG-2013-4320,05/11/2013,06/11/2013,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical,Deborah Brumfield,None
5,22732,IN-2013-42360,28/06/2013,01/07/2013,Second Class,JM-15655,Jim Mitchum,Corporate,Sydney,New South Wales,...,Phones,"Samsung Smart Phone, with Caller ID",2862.675,5,0.1,763.2750,897.35,Critical,Anthony Jacobs,None
6,30570,IN-2011-81826,07/11/2011,09/11/2011,First Class,TS-21340,Toby Swindell,Consumer,Porirua,Wellington,...,Chairs,"Novimex Executive Leather Armchair, Adjustable",1822.080,4,0.0,564.8400,894.77,Critical,Anthony Jacobs,None
7,31192,IN-2012-86369,14/04/2012,18/04/2012,Standard Class,MB-18085,Mick Brown,Consumer,Hamilton,Waikato,...,Tables,"Chromcraft Conference Table, Fully Assembled",5244.840,6,0.0,996.4800,878.38,High,Anthony Jacobs,None
8,40155,CA-2014-135909,14/10/2014,21/10/2014,Standard Class,JW-15220,Jane Waco,Corporate,Sacramento,California,...,Binders,Fellowes PB500 Electric Punch Plastic Comb Bin...,5083.960,5,0.2,1906.4850,867.69,Low,Matt Collister,None
9,40936,CA-2012-116638,28/01/2012,31/01/2012,Second Class,JH-15985,Joseph Holt,Consumer,Concord,North Carolina,...,Tables,Chromcraft Bull-Nose Wood Oval Conference Tabl...,4297.644,13,0.4,-1862.3124,865.74,Critical,Chuck Magee,None


# Preprocessing

In [12]:
# Check for null values
superstore_df[superstore_df.notnull()].count()


row_id            51295
order_id          51295
order_date        51295
ship_date         51295
ship_mode         51295
customer_id       51295
customer_name     51295
segment           51295
city              51295
state             51295
country           51295
postal_code        9996
market            51295
region            51295
product_id        51295
category          51295
sub_category      51295
product_name      51295
sales             51295
quantity          51295
discount          51295
profit            51295
shipping_cost     51295
order_priority    51295
person            46266
return             3055
dtype: int64

In [13]:
# Look at general statistics and size
print(f'Data size: {superstore_df.shape}')
superstore_df.describe()


Data size: (51295, 26)


,row_id,sales,quantity,discount,profit,shipping_cost
count,51295.000000,51295.000000,51295.000000,51295.000000,51295.000000,51295.000000
mean,25644.896598,246.481137,3.476518,0.142905,28.610120,26.374345
std,14806.333027,487.544207,2.278712,0.212272,174.333574,57.294351
min,1.000000,0.444000,1.000000,0.000000,-6599.978000,0.000000
25%,12821.500000,30.761250,2.000000,0.000000,0.000000,2.610000
50%,25645.000000,85.056000,3.000000,0.000000,9.240000,7.790000
75%,38466.500000,251.038800,5.000000,0.200000,36.810000,24.450000
max,51290.000000,22638.480000,14.000000,0.850000,8399.976000,933.570000


In [14]:
# Look at the data types of the df
superstore_df.dtypes

row_id              int64
order_id           object
order_date         object
ship_date          object
ship_mode          object
customer_id        object
customer_name      object
segment            object
city               object
state              object
country            object
postal_code        object
market             object
region             object
product_id         object
category           object
sub_category       object
product_name       object
sales             float64
quantity            int64
discount          float64
profit            float64
shipping_cost     float64
order_priority     object
person             object
return             object
dtype: object

In [16]:
# Modify datatypes of Order and Ship Date to datetime

superstore_df['order_date']=pd.to_datetime(superstore_df['order_date'])
superstore_df['ship_date']=pd.to_datetime(superstore_df['ship_date'])
superstore_df.dtypes

row_id                     int64
order_id                  object
order_date        datetime64[ns]
ship_date         datetime64[ns]
ship_mode                 object
customer_id               object
customer_name             object
segment                   object
city                      object
state                     object
country                   object
postal_code               object
market                    object
region                    object
product_id                object
category                  object
sub_category              object
product_name              object
sales                    float64
quantity                   int64
discount                 float64
profit                   float64
shipping_cost            float64
order_priority            object
person                    object
return                    object
dtype: object

In [17]:
#Get day of the week from Order Date column (0=monday - 6=Sunday)

superstore_df['week_day']=pd.DatetimeIndex(superstore_df['order_date']).dayofweek
superstore_df.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,...,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,person,return,week_day
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical,Kelly Williams,None,1
1,26341,IN-2013-77878,2013-05-02,2013-07-02,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical,Anthony Jacobs,Yes,3
2,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium,Anthony Jacobs,None,3
3,13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium,Anna Andreadi,None,0
4,47221,SG-2013-4320,2013-05-11,2013-06-11,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical,Deborah Brumfield,None,5


In [18]:
#Get month from Order Date column (1=january - 12=December)

superstore_df['Month_number']=pd.DatetimeIndex(superstore_df['order_date']).month
superstore_df.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,...,sales,quantity,discount,profit,shipping_cost,order_priority,person,return,week_day,Month_number
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,2309.650,7,0.0,762.1845,933.57,Critical,Kelly Williams,None,1,7
1,26341,IN-2013-77878,2013-05-02,2013-07-02,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,3709.395,9,0.1,-288.7650,923.63,Critical,Anthony Jacobs,Yes,3,5
2,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,5175.171,9,0.1,919.9710,915.49,Medium,Anthony Jacobs,None,3,10
3,13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,2892.510,5,0.1,-96.5400,910.16,Medium,Anna Andreadi,None,0,1
4,47221,SG-2013-4320,2013-05-11,2013-06-11,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,2832.960,8,0.0,311.5200,903.04,Critical,Deborah Brumfield,None,5,5


In [19]:
#Classify rows into profitable (1) and non-profitable (0)

superstore_df['profit_classification']= np.where(superstore_df['profit']<=0, 0, 1)
superstore_df

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,...,quantity,discount,profit,shipping_cost,order_priority,person,return,week_day,Month_number,profit_classification
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,7,0.0,762.1845,933.57,Critical,Kelly Williams,None,1,7,1
1,26341,IN-2013-77878,2013-05-02,2013-07-02,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,9,0.1,-288.7650,923.63,Critical,Anthony Jacobs,Yes,3,5,0
2,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,9,0.1,919.9710,915.49,Medium,Anthony Jacobs,None,3,10,1
3,13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,5,0.1,-96.5400,910.16,Medium,Anna Andreadi,None,0,1,0
4,47221,SG-2013-4320,2013-05-11,2013-06-11,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,8,0.0,311.5200,903.04,Critical,Deborah Brumfield,None,5,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51290,29002,IN-2014-62366,2014-06-19,2014-06-19,Same Day,KE-16420,Katrina Edelman,Corporate,Kure,Hiroshima,...,5,0.0,4.5000,0.01,Medium,Shirley Daniels,Yes,3,6,1
51291,35398,US-2014-102288,2014-06-20,2014-06-24,Standard Class,ZC-21910,Zuschuss Carroll,Consumer,Houston,Texas,...,1,0.8,-1.1100,0.01,Medium,Anna Andreadi,None,4,6,0
51292,40470,US-2013-155768,2013-02-12,2013-02-12,Same Day,LB-16795,Laurel Beltran,Home Office,Oxnard,California,...,3,0.0,11.2308,0.01,High,Matt Collister,None,1,2,1
51293,9596,MX-2012-140767,2012-02-18,2012-02-22,Standard Class,RB-19795,Ross Baird,Home Office,Valinhos,São Paulo,...,2,0.0,2.4000,0.00,Medium,Chuck Magee,None,5,2,1


In [20]:
# Order Dataframe by order date
superstore_df=superstore_df.sort_values('order_date',ascending=True)
superstore_df.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,...,quantity,discount,profit,shipping_cost,order_priority,person,return,week_day,Month_number,profit_classification
22971,22253,IN-2011-47883,2011-01-01,2011-08-01,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,...,3,0.1,36.036,9.72,Medium,Anthony Jacobs,None,5,1,1
42074,22254,IN-2011-47883,2011-01-01,2011-08-01,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,...,2,0.1,15.342,1.80,Medium,Anthony Jacobs,None,5,1,1
31889,22255,IN-2011-47883,2011-01-01,2011-08-01,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,...,5,0.1,37.770,4.70,Medium,Anthony Jacobs,None,5,1,1
25105,48883,HU-2011-1220,2011-01-01,2011-05-01,Second Class,AT-735,Annie Thurman,Consumer,Budapest,Budapest,...,4,0.0,29.640,8.17,High,None,None,5,1,1
9358,42433,AG-2011-2040,2011-01-01,2011-06-01,Standard Class,TB-11280,Toby Braunhardt,Consumer,Constantine,Constantine,...,2,0.0,106.140,35.46,Medium,Deborah Brumfield,None,5,1,1


# Transform data to fit into Machine Learning Model

In [22]:
# Drop unnecesary columns

superstore_ML_df=superstore_df.drop(columns=['row_id',
                                          'order_id',
                                          'customer_id',
                                          'customer_name',
                                          'postal_code',
                                          'product_id',
                                          'product_name',
                                          'ship_date',
                                          'city',
                                          'state',
                                          'country',
                                          'order_date',
                                          'sub_category',
                                          'market',
                                          'profit',
                                            'person',
                                             'return'                                        
                                         ])
superstore_ML_df.head()

,ship_mode,segment,region,category,sales,quantity,discount,shipping_cost,order_priority,week_day,Month_number,profit_classification
22971,Standard Class,Consumer,Oceania,Office Supplies,120.366,3,0.1,9.72,Medium,5,1,1
42074,Standard Class,Consumer,Oceania,Office Supplies,55.242,2,0.1,1.80,Medium,5,1,1
31889,Standard Class,Consumer,Oceania,Furniture,113.670,5,0.1,4.70,Medium,5,1,1
25105,Second Class,Consumer,EMEA,Office Supplies,66.120,4,0.0,8.17,High,5,1,1
9358,Standard Class,Consumer,Africa,Office Supplies,408.300,2,0.0,35.46,Medium,5,1,1


In [23]:
superstore_categories = superstore_ML_df.dtypes[superstore_ML_df.dtypes == "object"].index.tolist()
superstore_categories

['ship_mode', 'segment', 'region', 'category', 'order_priority']

In [24]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(superstore_ML_df[superstore_categories]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(superstore_categories)
encode_df.head()

,ship_mode_First Class,ship_mode_Same Day,ship_mode_Second Class,ship_mode_Standard Class,segment_Consumer,segment_Corporate,segment_Home Office,region_Africa,region_Canada,region_Caribbean,...,region_South,region_Southeast Asia,region_West,category_Furniture,category_Office Supplies,category_Technology,order_priority_Critical,order_priority_High,order_priority_Low,order_priority_Medium
0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [25]:
# Merge one-hot encoded features and drop the originals
superstore_ML_df = superstore_ML_df.merge(encode_df,left_index=True, right_index=True)
superstore_ML_df = superstore_ML_df.drop(superstore_categories,1)
superstore_ML_df.head()

,sales,quantity,discount,shipping_cost,week_day,Month_number,profit_classification,ship_mode_First Class,ship_mode_Same Day,ship_mode_Second Class,...,region_South,region_Southeast Asia,region_West,category_Furniture,category_Office Supplies,category_Technology,order_priority_Critical,order_priority_High,order_priority_Low,order_priority_Medium
22971,120.366,3,0.1,9.72,5,1,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
42074,55.242,2,0.1,1.80,5,1,1,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
31889,113.670,5,0.1,4.70,5,1,1,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
25105,66.120,4,0.0,8.17,5,1,1,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
9358,408.300,2,0.0,35.46,5,1,1,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [27]:
X=superstore_ML_df.copy()
X=superstore_ML_df.drop(columns=['profit_classification'])
X.head()

,sales,quantity,discount,shipping_cost,week_day,Month_number,ship_mode_First Class,ship_mode_Same Day,ship_mode_Second Class,ship_mode_Standard Class,...,region_South,region_Southeast Asia,region_West,category_Furniture,category_Office Supplies,category_Technology,order_priority_Critical,order_priority_High,order_priority_Low,order_priority_Medium
22971,120.366,3,0.1,9.72,5,1,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
42074,55.242,2,0.1,1.80,5,1,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
31889,113.670,5,0.1,4.70,5,1,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
25105,66.120,4,0.0,8.17,5,1,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
9358,408.300,2,0.0,35.46,5,1,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [29]:
y=superstore_ML_df['profit_classification']
y

22971    1
42074    1
31889    1
25105    1
9358     1
        ..
49371    1
25319    1
35497    1
13965    1
50923    0
Name: profit_classification, Length: 51295, dtype: int32

# Train and test Machine learning

In [30]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=25)

# Creating StandardScaler instance
scaler = StandardScaler()

# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Logistic Regression

In [31]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=100,
                                random_state=25)

In [32]:
classifier.fit(X_train_scaled, y_train)

LogisticRegression(random_state=25)

In [33]:
y_pred = classifier.predict(X_test_scaled)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(20)

,Prediction,Actual
0,1,1
1,1,1
2,1,1
3,1,1
4,0,0
5,0,0
6,1,1
7,1,1
8,1,1
9,1,1


In [39]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,2558,768
Actual 1,226,9272


In [40]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, y_pred)
print(acc_score)

0.9224890829694323


In [43]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, y_pred))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,2558,768
Actual 1,226,9272


Accuracy Score : 0.9224890829694323
Classification Report
              precision    recall  f1-score   support

           0       0.92      0.77      0.84      3326
           1       0.92      0.98      0.95      9498

    accuracy                           0.92     12824
   macro avg       0.92      0.87      0.89     12824
weighted avg       0.92      0.92      0.92     12824

